<a href="https://colab.research.google.com/github/BryanDYang/MOSA-Fall-Hackathon-2024/blob/main/Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries
import yfinance as yf
import pandas as pd
import numpy as np
import json

import requests
from datetime import datetime, timedelta
from textblob import TextBlob

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Mount the google drive
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


# 1. Research & Prototype using WRDS

Goal: Begin with a small, manageable dataset and build a proof-of-concept (PoC) to validate the feasibility of the ESG-based recommendation approach. This step helps ensure that our planned APIs, data models, and algorithms are compatible, reliable, and useful before moving on to a larger-scale MVP.

In [ ]:
!pip install wrds

In [3]:
# Using the Wharton Research Data Services
import wrds
conn = wrds.Connection(wrds_username='bryanyang')
conn.create_pgpass_file()

Loading library list...
Done
Enter your WRDS username [bryanyang]:bryanyang
Enter your password:··········


In [4]:
# Listing all the libraries of WRDS
conn.list_libraries().sort()
conn.list_libraries()

['aha',
 'aha_hcris_3years',
 'aha_hcris_recent',
 'aha_it_survey_3years',
 'aha_it_survey_recent',
 'aha_sample',
 'ahasamp',
 'audit',
 'audit_acct_os',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'audit_europe',
 'audit_oia',
 'audit_sedar',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex',
 'boardex_eur',
 'boardex_na',
 'boardex_row',
 'boardex_trial',
 'boardex_uk',
 'boardsmp',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_ap_ar',
 'calcbench_business_combinations',
 'calcbench_business_segments',
 'calcbench_commitment_contingency',
 'calcbench_common',
 'calcbench_derivatives_hedging',
 'calcbench_discontinued_ops',
 'calcbench_equity_compensation',
 'calcbench_equity_investments',
 'calcbench_equity_repurchases',
 'calcbench_fair_value',
 'calcbench_goodwill',
 'calcbench_income_tax',
 'calcbench_inventory',
 'calcbench_leases',


In [ ]:
# Exploring msci esg tables.
conn.list_tables(library='msci_esg')

['msci_esg_dictionary', 'msci_esg_ratings']

In [ ]:
# Extracting first 5 rows of msci_esg rating
msci_esg_dict = conn.get_table(library='msci_esg', table='msci_esg_dictionary', obs=5)
msci_esg_dict

,index,esg_category,factor_name,description,column_header,data_type,start_date
0,1,Other,Issuer Name,System factor,ISSUER_NAME,String [Example Text],2007-01-01
1,2,Issuer Data,Issuer Id,Issuer Id,ISSUERID,String [Example Text],2007-01-01
2,3,Other,Issuer Ticker,System factor,ISSUER_TICKER,String [Example Text],2007-01-01
3,4,Issuer Data,Issuer CUSIP,Issuer CUSIP. System factor,ISSUER_CUSIP,String [Example Text],2007-01-01
4,5,Issuer Data,Issuer SEDOL,Issuer SEDOL. System factor,ISSUER_SEDOL,String [Example Text],2007-01-01


In [ ]:
# Extracting first 5 rows of msci_esg rating
msci_esg_rt = conn.get_table(library='msci_esg', table='msci_esg_ratings', obs=5)
msci_esg_rt

,issuer_name,issuerid,issuer_ticker,issuer_cusip,issuer_sedol,issuer_isin,issuer_cntry_domicile,as_of_date,iva_industry,iva_rating_date,...,board_majority_ind_of_other_int,board_majority_ind_other_int_sd,no_nom_committee,no_nom_committee_sd,nom_committee_chair_ind,nom_committee_chair_ind_sd,nom_committee_ind,nom_committee_ind_sd,corp_behav_tax_transp_score,extern_manage_co
0,COBALTS Trust for Sprint Capital Notes,IID000000001789005,None,19074VAA0,None,US19074VAA08,US,2013-01-01,Wireless Telecommunication Services,2012-02-01,...,None,None,None,None,None,None,None,None,None,None
1,COBALTS Trust for Sprint Capital Notes,IID000000001789005,None,19074VAA0,None,US19074VAA08,US,2013-02-01,Wireless Telecommunication Services,2012-02-01,...,None,None,None,None,None,None,None,None,None,None
2,COBALTS Trust for Sprint Capital Notes,IID000000001789005,None,19074VAA0,None,US19074VAA08,US,2013-03-01,Wireless Telecommunication Services,2012-02-01,...,None,None,None,None,None,None,None,None,None,None
3,COBALTS Trust for Sprint Capital Notes,IID000000001789005,None,19074VAA0,None,US19074VAA08,US,2013-10-01,Wireless Telecommunication Services,2013-03-26,...,None,None,None,None,None,None,None,None,None,None
4,COBALTS Trust for Sprint Capital Notes,IID000000001789005,None,19074VAA0,None,US19074VAA08,US,2013-11-01,Wireless Telecommunication Services,2013-03-26,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# Exploring msci esg tables.
conn.list_tables(library='msci_esg_samp')

['msci_esg_expanded', 'msci_esg_ratings', 'msci_esg_summary']

In [ ]:
# Exploring msci esg tables.
conn.list_tables(library='msciesg')

['msci_esg_dictionary',
 'msci_esg_expanded',
 'msci_esg_ratings',
 'msci_esg_summary']

In [ ]:
# Extracting first 5 rows of msciesg expanded
msci_esg_exp = conn.get_table(library='msciesg', table='msci_esg_dictionary', obs=5)
msci_esg_exp

,index,esg_category,factor_name,description,column_header,data_type,start_date
0,1,Other,Issuer Name,System factor,ISSUER_NAME,String [Example Text],2007-01-01
1,2,Issuer Data,Issuer Id,Issuer Id,ISSUERID,String [Example Text],2007-01-01
2,3,Other,Issuer Ticker,System factor,ISSUER_TICKER,String [Example Text],2007-01-01
3,4,Issuer Data,Issuer CUSIP,Issuer CUSIP. System factor,ISSUER_CUSIP,String [Example Text],2007-01-01
4,5,Issuer Data,Issuer SEDOL,Issuer SEDOL. System factor,ISSUER_SEDOL,String [Example Text],2007-01-01


Strange that there are no tables called `msci_esg_expaneded` & `msci_esg_summary` within `msciesg` library

In [ ]:
# Exploring sp esg tables.
conn.list_tables(library='sp_esg')

['spgquestion', 'wrds_esg', 'wrds_esg_facts']

In [ ]:
# Extrating first 5 rows of sp_esg question table
sp_esg_q = conn.get_table(library='sp_esg', table='spgquestion', obs=5)
sp_esg_q

,questionid,questionname,questiondescription
0,1,Absentee Rate,Please indicate your company's absentee rate f...
1,2,Absenteeism: Work-Related Stress,None
2,3,Access for Disabled People,Please briefly describe initiatives that your ...
3,4,Access Through Education,Please briefly describe initiatives undertaken...
4,5,Access to Genetic Resources,Do you have policies and standards regarding t...


In [ ]:
# Extrating first 5 rows of sp_esg wrds_esg
sp_esg_wrds = conn.get_table(library='sp_esg', table='wrds_esg', obs=5)
sp_esg_wrds

,scoreid,dimensionscoreid,criteriascoreid,questionscoreid,institutionid,assessmentyear,scoredate,mostrecentsequence,csaindustrymapid,csaindustryname,...,msacasedate,msacasetitle,msacasescore,msacaseimpactlevelname,msacaseimpactrating,msacaseresponsename,msacaseresponserating,msaimpactid,msaimpactdate,criteriascoreimpact
0,5A1E6322-1057-42E2-BEBB-DED5A53D6884,BF23BB3C-1169-42C0-A1AD-41A1DC65CBFA,None,None,15333,2022,2023-03-17,1,C0F2A1B7-C793-4320-BEB1-C54524337F22,INS Insurance,...,None,None,None,None,None,None,None,None,None,None
1,5A1E6322-1057-42E2-BEBB-DED5A53D6884,BF23BB3C-1169-42C0-A1AD-41A1DC65CBFA,B8E71257-155C-44F6-8D8E-144D0DE170DD,D54A2853-685E-4CEA-9A86-341F4E298DEE,15333,2022,2023-03-17,1,C0F2A1B7-C793-4320-BEB1-C54524337F22,INS Insurance,...,None,None,None,None,None,None,None,None,None,None
2,5A1E6322-1057-42E2-BEBB-DED5A53D6884,BF23BB3C-1169-42C0-A1AD-41A1DC65CBFA,B8E71257-155C-44F6-8D8E-144D0DE170DD,B53F283D-BF69-465E-83D4-B28627DFD38F,15333,2022,2023-03-17,1,C0F2A1B7-C793-4320-BEB1-C54524337F22,INS Insurance,...,None,None,None,None,None,None,None,None,None,None
3,5A1E6322-1057-42E2-BEBB-DED5A53D6884,BF23BB3C-1169-42C0-A1AD-41A1DC65CBFA,B8E71257-155C-44F6-8D8E-144D0DE170DD,80B48B9F-AB6C-413C-A96E-C393116F25A2,15333,2022,2023-03-17,1,C0F2A1B7-C793-4320-BEB1-C54524337F22,INS Insurance,...,None,None,None,None,None,None,None,None,None,None
4,5A1E6322-1057-42E2-BEBB-DED5A53D6884,BF23BB3C-1169-42C0-A1AD-41A1DC65CBFA,B8E71257-155C-44F6-8D8E-144D0DE170DD,194F91BA-6B32-45EC-9475-C3B68A9EE782,15333,2022,2023-03-17,1,C0F2A1B7-C793-4320-BEB1-C54524337F22,INS Insurance,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
sp_esg_wrds.shape

(5, 41)

In [ ]:
# Extrating first 5 rows of sp_esg wrds_esg_facts
sp_esg_wrds_f = conn.get_table(library='sp_esg', table='wrds_esg_facts', obs=5)
sp_esg_wrds_f

,institutionid,scoreid,dimensionscoreid,criteriascoreid,questionscoreid,factreplyyear,questionname,factname,factdetail,factdetaildisplay,...,factreplyraw,factreplycomments,factreplyrawcalculated,reportedunit,factreplycurrency,factsourcename,questionfactspublic,questionfactspartpublic,questionfactsshared,questionfactsflexible
0,4170432,DD019033-A598-437A-838F-774129EB2822,E904D5E8-AAB8-4BFA-8905-31CA9A473F0B,9A115C6F-0597-439B-BBBC-DBD6524F8D31,00000335-988C-412B-956D-0C7E54EA898F,2022,People Analytics,TAR / People Analytics / Strategic Workforce P...,None,Use of People Analytics in strategic workforce...,...,YES,None,0,None,USD,SAM,0,0,1,0
1,4170432,DD019033-A598-437A-838F-774129EB2822,E904D5E8-AAB8-4BFA-8905-31CA9A473F0B,9A115C6F-0597-439B-BBBC-DBD6524F8D31,00000335-988C-412B-956D-0C7E54EA898F,2022,People Analytics,TAR / People Analytics / Organizational Networ...,None,Use of People Analytics for organizational net...,...,NO,None,0,None,USD,SAM,0,0,1,0
2,4170432,DD019033-A598-437A-838F-774129EB2822,E904D5E8-AAB8-4BFA-8905-31CA9A473F0B,9A115C6F-0597-439B-BBBC-DBD6524F8D31,00000335-988C-412B-956D-0C7E54EA898F,2022,People Analytics,TAR / People Analytics / Selection,None,Company's Use of People Analytics (PA),...,Applicable,None,0,None,USD,SAM,0,0,1,0
3,4170432,DD019033-A598-437A-838F-774129EB2822,E904D5E8-AAB8-4BFA-8905-31CA9A473F0B,9A115C6F-0597-439B-BBBC-DBD6524F8D31,00000335-988C-412B-956D-0C7E54EA898F,2022,People Analytics,TAR / People Analytics / Measuring Employee Pe...,None,Use of People Analytics for measuring employee...,...,YES,None,0,None,USD,SAM,0,0,1,0
4,4170432,DD019033-A598-437A-838F-774129EB2822,E904D5E8-AAB8-4BFA-8905-31CA9A473F0B,9A115C6F-0597-439B-BBBC-DBD6524F8D31,00000335-988C-412B-956D-0C7E54EA898F,2022,People Analytics,TAR / People Analytics / Flight Risks,None,Use of People Analytics for flight risks analysis,...,YES,None,0,None,USD,SAM,0,0,1,0


In [ ]:
sp_esg_wrds_f.shape

(5, 21)

In [ ]:
# Exploring tr esg tables.
conn.list_tables(library='tresgsmp')

['wrds_ref_esg']

In [ ]:
# Exploring tr esg tables.
conn.list_tables(library='tresg')

['esgasrepdata',
 'esgcode',
 'esgdesc',
 'esgdniratings',
 'esgenvdatapoint',
 'esgenvindicator',
 'esggovdatapoint',
 'esggovindicator',
 'esgitem',
 'esgorgindcls',
 'esgscores',
 'esgsocdatapoint',
 'esgsocindicator',
 'esgsourcedata',
 'esgsourcemap',
 'esgstmtdet',
 'wrds_ref_esg',
 'wrds_ref_esg_item']

In [ ]:
# Exploring tr esg tables.
conn.list_tables(library='tr_esg')

['esgasrepdata',
 'esgcode',
 'esgdesc',
 'esgdniratings',
 'esgenvdatapoint',
 'esgenvindicator',
 'esggovdatapoint',
 'esggovindicator',
 'esgitem',
 'esgorgindcls',
 'esgscores',
 'esgsocdatapoint',
 'esgsocindicator',
 'esgsourcedata',
 'esgsourcemap',
 'esgstmtdet',
 'wrds_ref_esg',
 'wrds_ref_esg_item']

In [ ]:
# Extrating data for tr_esg ratings table
tr_esg_ratings = conn.get_table(library='tr_esg', table='esgdniratings')
unique_values = tr_esg_ratings['orgpermid'].unique()

# Counting the unique company ids with esg ratings
unique_count = len(unique_values)
unique_count

12562

In [ ]:
# Extrating data for tr_esg ratings table
tr_esg_scores = conn.get_table(library='tr_esg', table='esgscores')
unique_values = tr_esg_scores['orgpermid'].unique()

# Counting the unique company ids with esg scores
unique_count = len(unique_values)
unique_count

14749

In [ ]:
# Extrating first 5 rows of data for tr_esg ratings table
tr_esg_rep_data = conn.get_table(library='tr_esg', table='esgasrepdata', obs=5)
tr_esg_rep_data

,orgpermid,item,fy,valueid,scalecode,unitcode,value_
0,4295613014,34,2016,505189817,None,None,47300000.0
1,4295613014,34,2017,505188870,None,None,48000000.0
2,4295613014,34,2018,505108579,None,None,47700000.0
3,4295613014,45,2013,505262753,None,None,0.0
4,4295613014,45,2014,505223532,None,None,0.0


In [ ]:
# Exploing comp tables
conn.list_tables(library='comp')

['aco_amda',
 'aco_imda',
 'aco_indfnta',
 'aco_indfntq',
 'aco_indfntytd',
 'aco_indsta',
 'aco_indstq',
 'aco_indstytd',
 'aco_notesa',
 'aco_notesq',
 'aco_notessa',
 'aco_notesytd',
 'aco_pnfnda',
 'aco_pnfndq',
 'aco_pnfndytd',
 'aco_pnfnta',
 'aco_pnfntq',
 'aco_pnfntytd',
 'aco_transa',
 'aco_transq',
 'aco_transsa',
 'aco_transytd',
 'adsprate',
 'asec_amda',
 'asec_imda',
 'asec_notesa',
 'asec_notesq',
 'asec_transa',
 'asec_transq',
 'bank_aacctchg',
 'bank_adesind',
 'bank_afnd1',
 'bank_afnd2',
 'bank_afnddc1',
 'bank_afnddc2',
 'bank_afntind',
 'bank_funda',
 'bank_funda_fncd',
 'bank_fundq',
 'bank_fundq_fncd',
 'bank_iacctchg',
 'bank_idesind',
 'bank_ifndq',
 'bank_ifndytd',
 'bank_ifntq',
 'bank_ifntytd',
 'bank_names',
 'bank_namesq',
 'chars',
 'co_aacctchg',
 'co_aaudit',
 'co_acthist',
 'co_adesind',
 'co_adjfact',
 'co_afnd1',
 'co_afnd2',
 'co_afnddc1',
 'co_afnddc2',
 'co_afntind1',
 'co_afntind2',
 'co_ainvval',
 'co_amkt',
 'co_busdescl',
 'co_cotype',
 'co_f

In [ ]:
# Querying some data from WRDS server
company = conn.get_table(library='comp', table='company', obs=10)
company

,conm,gvkey,add1,add2,add3,add4,addzip,busdesc,cik,city,...,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate,curr_sp500_flag
0,A & E PLASTIK PAK INC,001000,None,None,None,None,None,A & E Plastik Pak Inc. is a commodity chemical...,None,None,...,3089,325,978,None,None,0,None,1978-06-30,None,0.0
1,A & M FOOD SERVICES INC,001001,1924 South Utica Avenue,None,None,None,94104,None,0000723576,Tulsa,...,5812,420,978,None,OK,0,None,1986-07-31,None,0.0
2,AAI CORP,001002,124 Industry Lane,None,None,None,21030-0126,"Textron Systems Corporation designs, develops,...",0001306124,Hunt Valley,...,3825,230,940,None,MD,0,www.textronsystems.com,1977-03-31,None,0.0
3,A.A. IMPORTING CO INC,001003,7700 Hall Street,None,None,None,63125,"A.A. Importing Company, Inc. designs, manufact...",0000730052,St. Louis,...,5712,449,976,None,MO,3,www.aaimporting.com,1992-04-30,None,0.0
4,AAR CORP,001004,"One AAR Place, 1100 North Wood Dale Road",None,None,None,60191,AAR Corp. provides products and services to co...,0000001750,Wood Dale,...,5080,110,925,B,IL,0,www.aarcorp.com,None,1972-04-24,0.0
5,A.B.A. INDUSTRIES INC,001005,10260 US Highway 19 North,None,None,None,33782,A.B.A. Industries Inc. was acquired by McSwain...,None,Pinellas Park,...,3724,110,925,None,FL,0,None,1983-01-31,None,0.0
6,ABC INDS INC,001006,301 Kings Highway,None,None,None,46590,"ABC Industries, Inc. manufactures and supplies...",None,Winona Lake,...,2711,415,976,None,IN,3,www.abc-industries.net,1983-06-30,None,0.0
7,ABKCO INDUSTRIES INC,001007,"1700 Broadway, Suite 41",None,None,None,10019,"ABKCO Music & Records, Inc. is a prominent ind...",0000001882,New York,...,3652,220,925,None,NY,3,www.abkco.com,1986-12-31,None,0.0
8,ABM COMPUTER SYSTEMS INC,001008,3 Whatney,None,None,None,92714,ABM Computer Systems Inc. was acquired by Fran...,None,Irvine,...,3577,188,940,None,CA,0,None,1987-10-31,None,0.0
9,ABS INDUSTRIES INC,001009,"Interstate Square, Suite 300",None,None,None,44904,"ABS Industries, Inc., through Colfor and Colma...",0000313368,Willoughby,...,3460,355,925,None,OH,3,None,1996-12-31,None,0.0


In [ ]:
company.shape

(10, 40)

# Step 1: Research & Prototype

Goal: Begin with a small, manageable dataset and build a proof-of-concept (PoC) to validate the feasibility of the ESG-based recommendation approach. This step helps ensure that our planned APIs, data models, and algorithms are compatible, reliable, and useful before moving on to a larger-scale MVP.

1. Identify Data Sources
- Open Sustainability Index API:

    Focus: Start by exploring the `emissions` and `targets` endpoints. Confirm that we can retrieve:

    Scope 1, 2, and 3 emissions data for a small set of companies.
    `sbt_status` and net-zero commitments to gauge ESG commitments.

    Action: Pick 5-10 companies (diverse in industry and size) and fetch their ESG metrics from the API.

    Deliverable: A JSON or CSV file containing ESG data for these target companies.

    https://www.opensustainabilityindex.org/api

In [ ]:
base_url = "https://api.opensustainabilityindex.org/v1"
osi_api_key = "demo"

def get_industries():
    # This endpoint returns a list of industries.
    url = f"{base_url}/industries"
    params = {
        "api-key": osi_api_key,
        "limit": 300,
        "offset": 0,
        "order": "desc"
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json().get("data", [])
    return data

def get_companies_by_industry(industry_name, limit=300, offset=0):
    # This function retrieves companies for a given industry name.
    url = f"{base_url}/companies"
    params = {
        "api-key": osi_api_key,
        "limit": limit,
        "offset": offset,
        "order": "desc",
        "industry": industry_name
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json().get("data", [])
    error = response.json().get("error", None)

    if error:
        # If the API returns an error field, handle it here
        print(f"Error retrieving companies for industry '{industry_name}': {error}")
        return []

    # "data" should represent companies now if the industry name is correct
    return data

def main():
    # Step 1: Get all industries
    industries = get_industries()

    # Step 2: Iterate over each industry and fetch its companies
    for industry in industries:
        industry_name = industry.get("name")
        company_count = industry.get("company_count", 0)

        print(f"\n=== Industry: {industry_name}")
        print(f"Company count: {company_count}")

        if company_count > 0:
            # Retrieve companies for this industry's name (not slug)
            companies = get_companies_by_industry(industry_name, limit=min(company_count, 300))
            if companies:
                for c in companies:
                    company_name = c.get("company_name", c.get("name", "Unknown"))
                    print(f"- {company_name}")
            else:
                print("No company data returned for this industry.")
        else:
            print("No companies listed for this industry.")

if __name__ == "__main__":
    main()



=== Industry: Aerospace and Defense
Company count: 27
- Saab AB
- CTT Systems AB
- Airbus

=== Industry: Air Freight Transportation and Logistics
Company count: 79
- PostNord AB
- DSV A / S

=== Industry: Air Transportation - Airlines
Company count: 29
No company data returned for this industry.

=== Industry: Air Transportation - Airport Services
Company count: 14
No company data returned for this industry.

=== Industry: Automobiles and Components
Company count: 293
- Volvo Car Group
- Volkswagen AG
- Toyota Motor Corporation
- Tesla Inc.
- Mercedes-Benz AG
- Ford Motor Company
- Bulten AB
- BMW Group
- Autoliv Inc

=== Industry: Banks, Diverse Financials, Insurance
Company count: 273
- Zurich Insurance Group Ltd
- UnitedHealth Group
- Tokio Marine Holdings, Inc.
- Swedbank AB
- Svenska Handelsbanken publ.
- Storskogen Group
- Shinhan Financial Group
- SBAB
- Sampo plc
- MetLife, Inc.
- KB Financial Group
- Intesa Sanpaolo
- ING Group
- If P&C Insurance Holding Ltd. (publ)
- EQT AB


In [ ]:
# Temporary workaround until we get the esg api key
# Downloading the static file directly from the Open Sustainability API website
esg_path = '/content/drive/Shareddrives/mosa_fall2024_hackathon/Data/'

with open(esg_path+"esg.json", "r") as f:
  esg_data = json.load(f)

esg_data

{'data': [{'company_name': 'Ørsted',
   'slug': 'orsted',
   'company_url': None,
   'industry': 'Electric Utilities and Independent Power Producers and Energy Traders (including fossil, alternative and nuclear energy)',
   'organization_type': 'Company',
   'tags': 'omx120',
   'commitment_type': 'BA1.5 Option 1',
   'status': 'Target set',
   'commitment_deadline': '2021-09-30',
   'total_reported_emission_scope_1_2_3': 7217000,
   'revenue': 11508,
   'hq_country': 'Denmark',
   'year': 2023,
   'currency': 'DKK',
   'emission_intensity': 627,
   'targets': [{'target_type': 'Intensity',
     'target_scope': '1+2+3',
     'target_year': 2040,
     'target': 'Long-term'},
    {'target_type': 'Intensity',
     'target_scope': '1+2',
     'target_year': 2040,
     'target': 'Long-term'},
    {'target_type': 'Absolute',
     'target_scope': '3',
     'target_year': 2040,
     'target': 'Long-term'},
    {'target_type': 'Intensity',
     'target_scope': '1+2',
     'target_year': 2025,
  

In [ ]:
# Convert esg_data to esg_df, a pandas dataframe
# Create a dataframe for main company-level data (excluding targets)
companies_df = pd.json_normalize(esg_data['data']).drop(columns=['targets'])
companies_df

,company_name,slug,company_url,industry,organization_type,tags,commitment_type,status,commitment_deadline,total_reported_emission_scope_1_2_3,revenue,hq_country,year,currency,emission_intensity
0,Ørsted,orsted,None,Electric Utilities and Independent Power Produ...,Company,omx120,BA1.5 Option 1,Target set,2021-09-30,7217000,11508.0,Denmark,2023,DKK,627.0
1,Zurich Insurance Group Ltd,zurich-insurance,None,"Banks, Diverse Financials, Insurance",Financial Institution,None,BA1.5 Option 1,Extended,2022-09-30,55119,41890.0,Switzerland,2022,USD,1.0
2,Xano Industri,xano-industri,None,None,None,None,None,None,None,8182,337.0,None,2022,USD,24.0
3,Wärtsilä Oyj,wartsila,None,None,None,omx120,None,None,None,2073903,6511.0,None,2023,EUR,319.0
4,WPP Plc,wpp,None,Media,Company,None,BA1.5 Option 1,Removed,2024-01-31,5161438,15404.0,United Kingdom,2020,GBP,335.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Shell,shell,None,None,None,None,None,None,None,57000000,323183.0,None,2023,USD,176.0
95,"Seven & i Holdings Co., Ltd.",seven-and-i-holdings,None,Food and Staples Retailing,None,None,Standard,Removed,2023-03-31,12618799,88030.0,Japan,2022,USD,143.0
96,Serneke Group AB (publ),serneke,None,None,None,None,None,None,None,92190,913.0,None,2022,SEK,101.0
97,SeenThis AB,seenthis,seenthis.co,Software and Services,SME,None,None,None,None,5907,11.0,Sweden,2022,SEK,537.0


In [ ]:
# Create a separate dataframe for targets
targets_df = pd.json_normalize(esg_data['data'], record_path = 'targets',
                               meta=[
                                 'company_name',
                                 'slug',
                                 'year',
                                 'hq_country'
                               ],
                               errors='ignore'
)
targets_df

,target_type,target_scope,target_year,target,company_name,slug,year,hq_country
0,Intensity,1+2+3,2040.0,Long-term,Ørsted,orsted,2023,Denmark
1,Intensity,1+2,2040.0,Long-term,Ørsted,orsted,2023,Denmark
2,Absolute,3,2040.0,Long-term,Ørsted,orsted,2023,Denmark
3,Intensity,1+2,2025.0,Near-term,Ørsted,orsted,2023,Denmark
4,Absolute,3,2032.0,Near-term,Ørsted,orsted,2023,Denmark
...,...,...,...,...,...,...,...,...
374,None,None,NaN,None,Serneke Group AB (publ),serneke,2022,None
375,Absolute,1+2,2030.0,Near-term,SeenThis AB,seenthis,2022,Sweden
376,Net-zero,1+2+3,2040.0,Net-zero,SeenThis AB,seenthis,2022,Sweden
377,Absolute,1+2+3,2040.0,Long-term,SeenThis AB,seenthis,2022,Sweden


## Explaining the ESG dataframe:

Splitting into Two DataFrames: One for companies, one for targets.

Since `target` is a list of target objects per comapny, we can separate it into its own dataframe using `pd.json_normalize()` and specifying `meta` fields from the parent level so that we can later join the targets back to the main data.

By normalizing the data into two distinct tables - one for company-level attributes and one for targets - we maintain a reltationl structure. Each target row is linked back to the parent company via an identifier (e.g., `comapny_name` or `slug`).

Our goal is to:
- Filter targets independently.
- Easily join the target back to the company data on shared keys.
- Aggregate, pivot, or perform complex transformation on targets without repeatedly parsing nested objects.

## What is `target` in the ESG dataframe?

In the esg data, the target refers to a company's climate or sustainability objectives - often emissions reduction goals, renewable energy commitments, or nest-zero pledges, typically tied to a centain date or criterion.

- Target Type: Whether it’s an absolute emissions reduction (lowering overall greenhouse gas output) or an intensity reduction (lowering emissions per unit of revenue or production).

- Target Scope: Which emissions scopes or categories are covered. For example:

    Scope 1: Direct emissions from company-owned sources.

    Scope 2: Indirect emissions from purchased electricity, steam, heating, and cooling.

    Scope 3: All other indirect emissions that occur in a company’s value chain.

- Target Year: The deadline by which the company intends to achieve this goal.

- Target Aim: Whether it’s a near-term goal (e.g., by 2025 or 2030), a long-term goal (e.g., by 2040), or a net-zero target.

- Financial APIs (e.g., Polygon.io, AlphaVantage, yahoo finance):

    Focus: Ensure we can pull historical stock price data or basic financial performance indicators for the same set of companies we gathered ESG data on.

    Action: Use a free tier (if available) or sample data endpoints to fetch:

    Daily closing prices over the last 1-3 months.
    
    Basic financial ratios (if accessible) like P/E ratios or market cap.

    Deliverable: A small CSV containing financial time series data aligned with the chosen companies.

In [ ]:
# Define the tickers and time range
tickers = ["MSFT", "UL", "ORSTED.CO"]
end_date = datetime.today().date()
start_date = end_date - timedelta(days=20) # roughly 3 months of data

# Fetch data
data = yf.download(tickers, start=start_date, end=end_date)

# Returned df will have a multi-index with levels for ticker and OHLC info
# Focusing on daily closing prices and flatten the df
closing_prices = data['Close'].reset_index()
closing_prices.columns = ['Date'] + tickers


[*********************100%***********************]  3 of 3 completed


In [ ]:
# Fetch basic financial ratios/info
info_records = []
for t in tickers:
    stock = yf.Ticker(t)
    info = stock.info
    # Extract some key metrics if available:
    record = {
        "Ticker": t,
        "ShortName": info.get("shortName", ""),
        "MarketCap": info.get("marketCap", None),
        "PE_Ratio": info.get("trailingPE", None),
        "ForwardPE": info.get("forwardPE", None),
        "DividendYield": info.get("dividendYield", None),
        "Sector": info.get("sector", ""),
        "Industry": info.get("industry", "")
    }
    info_records.append(record)

financial_df = pd.DataFrame(info_records)


In [ ]:
closing_prices

,Date,MSFT,UL,ORSTED.CO
0,2024-11-29,423.459991,391.799988,59.840000
1,2024-12-02,430.980011,381.399994,59.910000
2,2024-12-03,431.200012,374.700012,59.380001
3,2024-12-04,437.420013,365.899994,59.369999
4,2024-12-05,442.619995,363.000000,59.549999
5,2024-12-06,443.570007,363.200012,59.040001
6,2024-12-09,446.019989,363.500000,58.840000
7,2024-12-10,443.329987,360.600006,58.709999
8,2024-12-11,448.989990,353.000000,58.779999
9,2024-12-12,449.559998,354.299988,58.709999


In [ ]:
financial_df

,Ticker,ShortName,MarketCap,PE_Ratio,ForwardPE,DividendYield,Sector,Industry
0,MSFT,Microsoft Corporation,3249265704960,36.058582,29.085247,0.0076,Technology,Software - Infrastructure
1,UL,Unilever PLC,145309155328,20.851450,18.524181,0.0329,Consumer Defensive,Household & Personal Products
2,ORSTED.CO,ORSTED A/S,139558387712,27.698082,14.328260,NaN,Utilities,Utilities - Renewable


In [ ]:
# Saving the results
closing_prices.to_json("esg_stocks_closing_prices.json", index=False)
financial_df.to_json("esg_stocks_financial_info.json", index=False)

- Sentiment Analysis (Optional for Prototype):

    Focus: This can be deferred since the first prototype might focus on ESG and basic financial data. But if we’d like a preview:

    Pull a few news headlines from a free market API or scrape recent ESG-related headlines.

    Use a simple NLP model (e.g., a publicly available sentiment library) to tag sentiment.

    Action (Optional): Annotate 5-10 news items for sentiment.

    Deliverable: A small table linking a company to sentiment score.

In [ ]:
# Mock data
data = [
    {"Company": "MSFT", "Headline": "Microsoft announces major investment in solar and wind energy projects."},
    {"Company": "MSFT", "Headline": "Microsoft is accused by activist groups of underreporting Scope 3 emissions."},
    {"Company": "UL", "Headline": "Unilever recognized for its industry-leading sustainable sourcing program."},
    {"Company": "UL", "Headline": "Unilever faces backlash over controversial palm oil suppliers."},
    {"Company": "ORSTED.CO", "Headline": "Ørsted expands offshore wind farms to meet Europe’s green energy targets."},
    {"Company": "ORSTED.CO", "Headline": "Ørsted’s supply chain challenges raise questions on project timelines."}
]

df = pd.DataFrame(data)

# Compute sentiment polarity
def get_sentiment_polarity(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

df["Sentiment_Score"] = df["Headline"].apply(get_sentiment_polarity)

# Optionally, classify sentiment as Positive/Neutral/Negative based on polarity
def classify_sentiment(score):
    if score > 0.1:
        return "Positive"
    elif score < -0.1:
        return "Negative"
    else:
        return "Neutral"

df["Sentiment_Class"] = df["Sentiment_Score"].apply(classify_sentiment)

df

,Company,Headline,Sentiment_Score,Sentiment_Class
0,MSFT,Microsoft announces major investment in solar ...,0.0625,Neutral
1,MSFT,Microsoft is accused by activist groups of und...,0.0000,Neutral
2,UL,Unilever recognized for its industry-leading s...,0.0000,Neutral
3,UL,Unilever faces backlash over controversial pal...,0.5500,Positive
4,ORSTED.CO,Ørsted expands offshore wind farms to meet Eur...,-0.2000,Negative
5,ORSTED.CO,Ørsted’s supply chain challenges raise questio...,0.0000,Neutral


2. Data Exploration and Validation
- Check Data Quality & Completeness:
  
    Verify that each chosen company has entries in both the ESG and financial datasets.
    
    Identify missing values or inconsistent data formats.

In [ ]:
slug_to_ticker = {
    "orsted": "ORSTED.CO",                        # Ørsted (Copenhagen)
    "zurich-insurance": None,                    # Zurich Insurance Group (SIX: ZURN)
    "xano-industri": None,                      # XANO Industri AB (OMX: XANO-B)
    "wartsila": None,                           # Wärtsilä Oyj Abp (HEL: WRT1V)
    "wpp": "WPP",                               # WPP PLC (NYSE: WPP or LSE: WPP)
    "world-courier-sweden": None,
    "wihlborgs-fastigheter": None,              # Wihlborgs Fastigheter AB (OMX: WIHL)
    "westpac-banking": "WBC.AX",                # Westpac Banking Corp. (ASX: WBC)
    "wells-fargo": "WFC",                       # Wells Fargo & Company (NYSE: WFC)
    "walmart": "WMT",                           # Walmart Inc. (NYSE: WMT)
    "wallenstam": None,                         # Wallenstam AB (OMX: WALL-B)
    "volvo-car": "VOLCAR-B.ST",                 # Volvo Car AB (Nasdaq Stockholm: VOLCAR B)
    "volkswagen": "VOW.DE",                     # Volkswagen AG (Xetra: VOW)
    "volati": None,                             # Volati AB (OMX: VOLO)
    "vodafone": "VOD",                          # Vodafone Group PLC ADR (NASDAQ: VOD)
    "vnv-global": None,                         # VNV Global AB (OMX: VNV)
    "vitrolife": None,                          # Vitrolife AB (OMX: VITR)
    "vitol": None,                              # Vitol is private, no ticker
    "visa": "V",                                # Visa Inc. (NYSE: V)
    "vimian": None,                             # Vimian Group AB (OMX: VIMIAN)
    "viaplay": None,                            # Viaplay Group AB (OMX: VPLAY-B)
    "vestum": None,                             # Vestum AB (OMX: VESTUM)
    "vestas-wind-systems": None,                # Vestas Wind Systems A/S (CPH: VWS)
    "verizon-communications": "VZ",             # Verizon Communications (NYSE: VZ)
    "veidekke-entreprenad": None,               # Veidekke ASA (OSL: VEI)
    "vattenfall": None,                         # Vattenfall is state-owned, no ticker
    "vasakronan": None,                         # Vasakronan is owned by Swedish pension funds, no public ticker
    "valmet": "VALMT.HE",                       # Valmet Oyj (HEL: VALMT)
    "valero-energy": "VLO",                     # Valero Energy Corporation (NYSE: VLO)
    "upm-kymmene": None,                        # UPM-Kymmene Oyj (HEL: UPM)
    "unitedhealth": "UNH",                      # UnitedHealth Group (NYSE: UNH)
    "united-parcel-service": "UPS",             # United Parcel Service (NYSE: UPS)
    "uniper-se": None,                          # Uniper SE (Xetra: UN01) - now majority owned by the German government
    "union-pacific": "UNP",                     # Union Pacific Corporation (NYSE: UNP)
    "unilever": "UL",                           # Unilever PLC ADR (NYSE: UL)
    "unicredit": None,                          # UniCredit S.p.A. (BIT: UCG)
    "ubs": None,                                # UBS Group AG (NYSE: UBS or SIX: UBSG)
    "tuifly-nordic": None,                      # TUIfly Nordic is part of TUI AG (Xetra: TUI1)
    "tryg": None,                               # Tryg A/S (CPH: TRYG)
    "truist-financial": "TFC",                  # Truist Financial Corporation (NYSE: TFC)
    "travelers": "TRV",                         # The Travelers Companies (NYSE: TRV)
    "toyota-motor": "TM",                       # Toyota Motor Corp. ADR (NYSE: TM)
    "totalenergies-se": "TTE",                  # TotalEnergies SE (NYSE: TTE)
    "tokio-marine-holdings": None,              # Tokio Marine Holdings Inc. (TYO: 8766)
    "tobii-dynavox": None,                      # Tobii Dynavox AB (OMX: TODX)
    "tietoevry": None,                          # TietoEVRY Oyj (HEL: TIETO)
    "thule": None,                              # Thule Group AB (OMX: THULE)
    "thermo-fisher-scientific": "TMO",          # Thermo Fisher Scientific Inc. (NYSE: TMO)
    "walt-disney": "DIS",                       # The Walt Disney Company (NYSE: DIS)
    "kroger": "KR",                             # The Kroger Co. (NYSE: KR)
    "home-depot": "HD",                         # The Home Depot (NYSE: HD)
    "economist": None,                          # The Economist Group is private
    "coca-cola": "KO",                          # The Coca-Cola Company (NYSE: KO)
    "tesla": "TSLA",                            # Tesla Inc. (NASDAQ: TSLA)
    "tencent-holdings": "TCEHY",                # Tencent Holdings ADR (OTC: TCEHY)
    "telia": None,                              # Telia Company AB (OMX: TELIA)
    "telenor": None,                            # Telenor ASA (OSL: TEL)
    "telefonica": "TEF",                        # Telefónica S.A. (NYSE: TEF)
    "tele2": None,                              # Tele2 AB (OMX: TEL2-B)
    "technipfmc": "FTI",                        # TechnipFMC plc (NYSE: FTI)
    "td-bank": "TD",                            # The Toronto-Dominion Bank (NYSE: TD)
    "target": "TGT",                            # Target Corporation (NYSE: TGT)
    "takeda-pharmaceutical": "TAK",             # Takeda Pharmaceutical Co. Ltd. ADR (NYSE: TAK)
    "taiwan-semiconductor": "TSM",              # Taiwan Semiconductor Manufacturing Co. ADR (NYSE: TSM)
    "systemair": None,                          # Systemair AB (OMX: SYSR)
    "sydbank": None,                            # Sydbank A/S (CPH: SYDB)
    "swedish-orphan-biovitrum": None,           # Swedish Orphan Biovitrum AB (OMX: SOBI)
    "swedbank": None,                           # Swedbank AB (OMX: SWED-A)
    "svenska-handelsbanken-publ": None,          # Svenska Handelsbanken AB (OMX: SHB-A)
    "svedbergs": None,                          # Svedbergs i Dalstorp AB (OMX: SVED-B)
    "suncor-energy": "SU",                      # Suncor Energy Inc. (NYSE: SU)
    "sun-life-financial": "SLF",                # Sun Life Financial Inc. (NYSE: SLF)
    "sumitomo-mitsui-financial": None,          # Sumitomo Mitsui Financial Group Inc. (NYSE: SMFG)
    "strawberry": None,                         # Strawberry is likely private
    "storytel": None,                           # Storytel AB (OMX: STORY-B)
    "storskogen": None,                         # Storskogen Group AB (publ) (OMX: STOR-B)
    "stora-enso": None,                         # Stora Enso Oyj (HEL: STERV)
    "stillfront": None,                         # Stillfront Group AB (OMX: SF)
    "stena-recycling": None,                    # Stena Recycling is private
    "stellantis": "STLA",                       # Stellantis N.V. (NYSE: STLA)
    "state-bank-of-india": "SBIN.NS",           # State Bank of India (NSE: SBIN)
    "ssab": None,                               # SSAB AB (OMX: SSAB-A)
    "spendrups-bryggeri": None,                 # Spendrups privately held
    "specialfastigheter-sverige": None,         # Specialfastigheter Sverige AB is state-owned, no ticker
    "southern": "SO",                           # The Southern Company (NYSE: SO)
    "sony": "SONY",                             # Sony Group Corp. ADR (NYSE: SONY)
    "softbank": "SFTBY",                        # SoftBank Group Corp. ADR (OTC: SFTBY)
    "sky": None,                                # Sky Group is owned by Comcast (NASDAQ: CMCSA)
    "skistar": None,                            # Skistar AB (OMX: SKIS-B)
    "skanska": None,                            # Skanska AB (OMX: SKA-B)
    "sk-hynix": None,                           # SK hynix Inc. (KRX: 000660)
    "sinopec": "SNP",                           # China Petroleum & Chemical Corp. ADR (NYSE: SNP)
    "siemens": None,                            # Siemens AG (Xetra: SIE)
    "shinhan-financial": "SHG",                 # Shinhan Financial Group Co. Ltd. ADR (NYSE: SHG)
    "shell": "SHEL",                            # Shell plc (NYSE: SHEL)
    "seven-and-i-holdings": None,               # Seven & i Holdings Co. Ltd. (TYO: 3382)
    "serneke": None,                            # Serneke Group AB (OMX: SRNKE-B)
    "seenthis": None,                           # SeenThis AB is private
    "securitas": None                           # Securitas AB (OMX: SECU-B)
}


In [ ]:
# Add a ticker column to ESG data based on the slug
companies_df['Ticker'] = companies_df['slug'].map(slug_to_ticker)

In [ ]:
merge_df = pd.merge(companies_df, financial_df, on='Ticker', how='inner')

In [ ]:
# Checking data quality & completeness once keys are matched:
# Companies that are in both dataframes
esg_tickers = set(companies_df['Ticker'].dropna())
fin_tickers = set(financial_df['Ticker'].dropna())

common_tickers = esg_tickers.intersection(fin_tickers)
print("Companies in both ESG and Financial datasets:", common_tickers)

only_in_esg = esg_tickers - fin_tickers
print("Companies only in ESG data:", only_in_esg)

only_in_fin = fin_tickers - esg_tickers
print("Companies only in Financial data:", only_in_fin)

Companies in both ESG and Financial datasets: {'ORSTED.CO', 'UL'}
Companies only in ESG data: {'SHEL', 'TM', 'VALMT.HE', 'TFC', 'TSM', 'VZ', 'VOLCAR-B.ST', 'DIS', 'SONY', 'VOD', 'UPS', 'TSLA', 'UNH', 'TAK', 'TCEHY', 'SHG', 'SNP', 'TD', 'FTI', 'TTE', 'WBC.AX', 'VLO', 'STLA', 'KO', 'KR', 'HD', 'WPP', 'SFTBY', 'WFC', 'SO', 'TRV', 'V', 'TMO', 'SBIN.NS', 'VOW.DE', 'UNP', 'WMT', 'TEF', 'SU', 'TGT', 'SLF'}
Companies only in Financial data: {'MSFT'}


In [ ]:
# For missing values:
print("ESG Missing Values:\n", companies_df.isna().sum())
print("Financial Missing Values:\n", financial_df.isna().sum())

ESG Missing Values:
 company_name                            0
slug                                    0
company_url                            76
industry                               39
organization_type                      41
tags                                   80
commitment_type                        54
status                                 54
commitment_deadline                    54
total_reported_emission_scope_1_2_3     0
revenue                                 5
hq_country                             39
year                                    0
currency                                0
emission_intensity                      5
Ticker                                 56
dtype: int64
Financial Missing Values:
 Ticker           0
ShortName        0
MarketCap        0
PE_Ratio         0
ForwardPE        0
DividendYield    1
Sector           0
Industry         0
dtype: int64


In [ ]:
#Check data types:
print("ESG dtypes:\n", companies_df.dtypes)
print("Financial dtypes:\n", financial_df.dtypes)

ESG dtypes:
 company_name                            object
slug                                    object
company_url                             object
industry                                object
organization_type                       object
tags                                    object
commitment_type                         object
status                                  object
commitment_deadline                     object
total_reported_emission_scope_1_2_3      int64
revenue                                float64
hq_country                              object
year                                     int64
currency                                object
emission_intensity                     float64
Ticker                                  object
dtype: object
Financial dtypes:
 Ticker            object
ShortName         object
MarketCap          int64
PE_Ratio         float64
ForwardPE        float64
DividendYield    float64
Sector            object
Industry          object
dt